In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# set larger font sizes
plt.rcParams.update({'font.size': 16})

In [ ]:

from matplotlib import font_manager
font_path = 'Times New Roman.ttf'
font_manager.fontManager.addfont(font_path)
plt.rcParams["font.family"] = "Times New Roman"

from adjustText import adjust_text
import re


In [4]:
industry_models = {
    # "TinyLlama-1.1b":{
    #     "W": 2048,
    #     "D": 22,
    #     "W/D": 2048/22,
    #     "N": 1.1e9,
    #     "Tgt N": "TinyLlama-1.1b"
    # },
    # "Llama-7b":{
    #     "W": 4096,
    #     "D": 32,
    #     "W/D": 128,
    #     "N": 7e9,
    #     "Tgt N":"Llama-7b"
    # },
    # "Llama-13b":{
    #     "W": 5120,
    #     "D": 40,
    #     "W/D": 5120/40,
    #     "N": 13e9,
    #     "Tgt N":"Llama-13b"
    # },
    # "Llama-33b":{
    #     "W": 6656,
    #     "D": 60,
    #     "W/D": 6656/60,
    #     "N": 33e9,
    #     "Tgt N":"Llama-33b"
    # },
    # "Llama-65b":{
    #     "W": 8192,
    #     "D": 80,
    #     "W/D": 8192/80,
    #     "N": 65e9,
    #     "Tgt N":"Llama-65b"
    # },
    # "Llama-2-7b":{
    #     "W": 4096,
    #     "D": 32,
    #     "W/D": 128,
    #     "N": 7e9,
    #     "Tgt N":"Llama-2-7b"
    # },
    "Llama-2-13b":{
        "W": 5120,
        "D": 40,
        "W/D": 5120/40,
        "N": 13e9,
        "Tgt N":"Llama-2-13b"
    },
    "Llama-2-34b":{
        "W": 6656,
        "D": 60,
        "W/D": 6656/60,
        "N": 34e9,
        "Tgt N":"Llama-2-34b"
    },
    # "Llama-2-70b":{
    #     "W": 8192,
    #     "D": 80,
    #     "W/D": 8192/80,
    #     "N": 70e9,
    #     "Tgt N":"Llama-2-70b"
    # },
    "Llama-3.1-8b":{
        "W": 4096,
        "D": 32,
        "W/D": 128,
        "N": 8e9,
        "Tgt N":"Llama-3.1-8b"
    },
    "Llama-3.1-70b":{
        "W": 8192,
        "D": 80,
        "W/D": 8192/80,
        "N": 70e9,
        "Tgt N":"Llama-3.1-70b"
    },
    "Llama-3.1-405b":{
        "W": 16394,
        "D": 126,
        "W/D": 16394/126,
        "N": 405e9,
        "Tgt N":"Llama-3.1-405b"
    },
    "Gemma-2b":{
        "W": 2048,
        "D": 18,
        "W/D": 2048/18,
        "N": 2e9,
        "Tgt N":"Gemma-2b"
    },
    "Gemma-7b":{
        "W": 3072,
        "D": 28,
        "W/D": 3072/28,
        "N": 7e9,
        "Tgt N":"Gemma-7b"
    },
    "Gemma-2-2b":{
        "W": 2304,
        "D": 26,
        "W/D": 2304/26,
        "N": 2e9,
        "Tgt N":"Gemma-2-2b"
    },
    "Gemma-2-9b":{
        "W": 3584,
        "D": 42,
        "W/D": 3584/42,
        "N": 9e9,
        "Tgt N":"Gemma-2-9b"
    },
    "Gemma-2-27b":{
        "W": 4608,
        "D": 46,
        "W/D": 4608/46,
        "N": 9e9,
        "Tgt N":"Gemma-2-27b"
    },
    # "Mistral-7b":{
    #     "W": 4096,
    #     "D": 32,
    #     "W/D": 4096/32,
    #     "N": 7e9,
    #     "Tgt N":"Mistral-7b"
    # },
    # "Phi-3.5-Mini":{
    #     "W": 3072,
    #     "D": 32,
    #     "W/D": 96,
    #     "N": 3.8e9,
    #     "Tgt N":"Phi-3.5-Mini"
    # },
    # "MiniCPM-V-2-2.4b":{
    #     "W": 2304,
    #     "D": 40,
    #     "W/D": 2304/40,
    #     "N": 2.8e9,
    #     "Tgt N":"MiniCPM-V-2-2.4b"
    # },
    # "MiniCPM-V-2-1.2b":{
    #     "W": 1536,
    #     "D": 52,
    #     "W/D": 1536/52,
    #     "N": 1.2e9,
    #     "Tgt N":"MiniCPM-V-2-1.2b"
    # },
    # "Deepseek-7b":{
    #     "W": 4096,
    #     "D": 30,
    #     "W/D": 4096/30,
    #     "N": 7e9,
    #     "Tgt N":"Deepseek-7b"
    # },
    # "Deepseek-67b":{
    #     "W": 8192,
    #     "D": 95,
    #     "W/D": 8192/95,
    #     "N": 67e9,
    #     "Tgt N":"Deepseek-67b"
    # },
}
special_models = dict(industry_models)
# w/d configurations from resolving scaling laws, all 0-4 "configs" are identical

#      width  depth  width_depth_ratio  params_active_precise
# 18    96.0    3.0          32.000000              5173248.0
# 13   128.0    4.0          32.000000              7503872.0
# 16   160.0    5.0          32.000000              9809920.0
# 14   224.0    6.0          37.333333             15597568.0
# 25   288.0    8.0          36.000000             22487040.0
# 22   320.0    9.0          35.555556             28672000.0
# 26   384.0   10.0          38.400000             37060608.0
# 23   480.0   12.0          40.000000             57384960.0
# 29   576.0   14.0          41.142857             84787200.0
# 31   640.0   15.0          42.666667            108462080.0
# 9    704.0   18.0          39.111111            149045248.0
# 6    832.0   21.0          39.619048            220872704.0
# 11  1024.0   23.0          44.521739            347078656.0
# 10  1120.0   26.0          43.076923            455311360.0
# 7   1312.0   26.0          50.461538            611958784.0
# 4   1504.0   30.0          50.133333            901726208.0

# format that as a dictionary

scaling_law_configs = {
    0: {"W": 96, "D": 3, "W/D": 32, "N": 5.173248e6, "Tgt N": "Porian et al."},
    1: {"W": 128, "D": 4, "W/D": 32, "N": 7.503872e6, "Tgt N": "Porian et al."},
    2: {"W": 160, "D": 5, "W/D": 32, "N": 9.809920e6, "Tgt N": "Porian et al."},
    3: {"W": 224, "D": 6, "W/D": 37.333333, "N": 1.5597568e7, "Tgt N": "Porian et al."},
    4: {"W": 288, "D": 8, "W/D": 36, "N": 2.2487040e7, "Tgt N": "Porian et al."},
    5: {"W": 320, "D": 9, "W/D": 35.555556, "N": 2.8672000e7, "Tgt N": "Porian et al."},
    6: {"W": 384, "D": 10, "W/D": 38.4, "N": 3.7060608e7, "Tgt N": "Porian et al."},
    7: {"W": 480, "D": 12, "W/D": 40, "N": 5.7384960e7, "Tgt N": "Porian et al."},
    8: {"W": 576, "D": 14, "W/D": 41.142857, "N": 8.4787200e7, "Tgt N": "Porian et al."},
    9: {"W": 640, "D": 15, "W/D": 42.666667, "N": 1.0846208e8, "Tgt N": "Porian et al."},
    10: {"W": 704, "D": 18, "W/D": 39.111111, "N": 1.49045248e8, "Tgt N": "Porian et al."},
    11: {"W": 832, "D": 21, "W/D": 39.619048, "N": 2.20872704e8, "Tgt N": "Porian et al."},
    12: {"W": 1024, "D": 23, "W/D": 44.521739, "N": 3.47078656e8, "Tgt N": "Porian et al."},
    13: {"W": 1120, "D": 26, "W/D": 43.076923, "N": 4.55311360e8, "Tgt N": "Porian et al."},
    14: {"W": 1312, "D": 26, "W/D": 50.461538, "N": 6.11958784e8, "Tgt N": "Porian et al."},
    15: {"W": 1504, "D": 30, "W/D": 50.133333, "N": 9.01726208e8, "Tgt N": "Porian et al."},
}

special_models.update(scaling_law_configs)

chinchilla_data = {
    "Parameters (million)": [44, 57, 74, 90, 106, 117, 140, 163, 175, 196, 217, 251, 278, 306, 425, 489, 509, 552, 587, 632, 
                             664, 724, 816, 893, 1018, 1143, 1266, 1424, 1429, 1593, 1609, 1731, 1794, 2007, 2283, 2298, 
                             2639, 2980, 3530, 3802, 4084, 4516, 6796, 9293, 11452, 12295, 12569, 13735, 14940, 16183],
    "d_model": [512, 576, 640, 640, 640, 768, 768, 768, 896, 896, 896, 1024, 1024, 1024, 1280, 1280, 1408, 1280, 1408, 
                1536, 1408, 1536, 1536, 1792, 1792, 1792, 2048, 2176, 2048, 2048, 2176, 2304, 2176, 2304, 2304, 2560, 
                2560, 2560, 2688, 2816, 2944, 3072, 3584, 4096, 4352, 4608, 4608, 4864, 4992, 5120],
    "ffw_size": [2048, 2304, 2560, 2560, 2560, 3072, 3072, 3072, 3584, 3584, 3584, 4096, 4096, 4096, 5120, 5120, 5632, 5120, 
                 5632, 6144, 5632, 6144, 6144, 7168, 7168, 7168, 8192, 8704, 8192, 8192, 8704, 9216, 8704, 9216, 9216, 
                 10240, 10240, 10240, 10752, 11264, 11776, 12288, 14336, 16384, 17408, 18432, 18432, 19456, 19968, 20480],
    "kv_size": [64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 
                128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 
                128, 128, 128, 128, 128],
    "n_heads": [8, 9, 10, 10, 10, 12, 12, 12, 14, 14, 14, 16, 16, 16, 10, 10, 11, 10, 11, 12, 11, 12, 12, 14, 14, 14, 16, 
                17, 16, 16, 17, 18, 17, 18, 18, 20, 20, 20, 22, 22, 22, 24, 28, 32, 32, 36, 32, 32, 32, 40],
    "n_layers": [8, 9, 10, 13, 16, 12, 15, 18, 14, 16, 18, 16, 18, 20, 18, 21, 18, 24, 21, 19, 24, 22, 25, 20, 23, 26, 22, 
                 22, 25, 28, 25, 24, 28, 28, 32, 26, 30, 34, 36, 36, 36, 36, 40, 42, 47, 44, 47, 47, 49, 47]
}
df = pd.DataFrame(chinchilla_data)
df["W"] = df["d_model"]
df["D"] = df["n_layers"]
df["W/D"] = df["d_model"] / df["n_layers"]
df["Tgt N"] = "Chinchilla"
df = df[["W", "D", "W/D", "Tgt N"]]
formatted_dict = {
    idx+16: {
        "W": row["W"],
        "D": row["D"],
        "W/D": row["W/D"],
        "Tgt N": row["Tgt N"]
    } for idx, row in df.iterrows()
}

special_models.update(formatted_dict)

# convert the models we ran into same format as the scaling law configs
our_models = {
    "50M-256-23": {
        "W": 256,
        "D": 23,
        "W/D": 11.13,
        "N": 50e6,
        "Tgt N": "50M-256-23"
    },
    "50M-256-27": {
        "W": 256,
        "D": 27,
        "W/D": 9.48,
        "N": 50e6,
        "Tgt N": "50M-256-27"
    },
    "100M-256-71": {
        "W": 256,
        "D": 71,
        "W/D": 3.61,
        "N": 100e6,
        "Tgt N": "100M-256-71"
    },
    "100M-256-80": {
        "W": 256,
        "D": 80,
        "W/D": 3.2,
        "N": 100e6,
        "Tgt N": "100M-256-80"
    },
    "100M-512-12": {
        "W": 512,
        "D": 12,
        "W/D": 42.67,
        "N": 100e6,
        "Tgt N": "100M-512-12"
    },
    "100M-512-13": {
        "W": 512,
        "D": 13,
        "W/D": 39.38,
        "N": 100e6,
        "Tgt N": "100M-512-13"
    },
    "100M-768-3": {
        "W": 768,
        "D": 3,
        "W/D": 256,
        "N": 100e6,
        "Tgt N": "100M-768-3"
    },
    "500M-1792-7": {
        "W": 1792,
        "D": 7,
        "W/D": 256,
        "N": 500e6,
        "Tgt N": "500M-1792-7"
    },
    "500M-1280-15": {
        "W": 1280,
        "D": 15,
        "W/D": 85.33,
        "N": 500e6,
        "Tgt N": "500M-1280-15"
    },
    "500M-768-45": {
        "W": 768,
        "D": 45,
        "W/D": 17.07,
        "N": 500e6,
        "Tgt N": "500M-768-45"
    },
    "1B-1792-18": {
        "W": 1792,
        "D": 18,
        "W/D": 99.56,
        "N": 1e9,
        "Tgt N": "1B-1792-18"
    },
    "1B-2560-8": {
        "W": 2560,
        "D": 8,
        "W/D": 320,
        "N": 1e9,
        "Tgt N": "1B-2560-8"
    },
    "1B-1280-36": {
        "W": 1280,
        "D": 36,
        "W/D": 35.56,
        "N": 1e9,
        "Tgt N": "1B-1280-36"
    },
    "2B-2048-27": {
        "W": 2048,
        "D": 27,
        "W/D": 75.56,
        "N": 2e9,
        "Tgt N": "2B-2048-27"
    },
    "2B-1536-50": {
        "W": 1536,
        "D": 50,
        "W/D": 30.72,
        "N": 2e9,
        "Tgt N": "2B-1536-50"
    },
    "2B-3072-12": {
        "W": 3072,
        "D": 12,
        "W/D": 256,
        "N": 2e9,
        "Tgt N": "2B-3072-12"
    },
    "2B-3072-12": {
        "W": 3072,
        "D": 12,
        "W/D": 256,
        "N": 2e9,
        "Tgt N": "2B-3072-12"
    },
    "500M-1024-28": {
        "W": 1024,
        "D": 28,
        "W/D": 36.57,
        "N": 5e8,
        "Tgt N": "500M-1024-28"
    },
    "70M-384-13": {
        "W": 384,
        "D": 13,
        "W/D": 29.5,
        "N": 7e7,
        "Tgt N": "70M-384-13"
    },
    "100M-384-36": {
        "W": 384,
        "D": 36,
        "W/D": 10.67,
        "N": 1e8,
        "Tgt N": "100M-384-36"
    },
    "100M-512-16": {
        "W": 512,
        "D": 16,
        "W/D": 32,
        "N": 1e8,
        "Tgt N": "100M-512-16"
    },
    "100M-512-11": {
        "W": 512,
        "D": 11,
        "W/D": 46.55,
        "N": 1e8,
        "Tgt N": "100M-512-16"
    },
    "100M-512-14": {
        "W": 512,
        "D": 14,
        "W/D": 36.57,
        "N": 1e8,
        "Tgt N": "100M-512-16"
    },
    # "4B-2816-32": {
    #     "W": 2816,
    #     "D": 32,
    #     "W/D": 88,
    #     "N": 4e9,
    #     "Tgt N": "4B-2816-32"
    # },
    # "7B-3584-36": {
    #     "W": 3584,
    #     "D": 36,
    #     "W/D": 99.56,
    #     "N": 7e9,
    #     "Tgt N": "7B-3584-36"
    # },
}

special_models.update(our_models)

In [5]:
def param_counter(width, depth, vocab_size, n_head, head_size, n_query_groups, intermediate_size):
    # Embedding layer parameters
    embedding_params = vocab_size * width
    
    # Attention parameters: attn + proj
    attn_shape = (n_head + 2 * n_query_groups) * head_size
    attn_params = (width * attn_shape) + (head_size * n_head * width)
    
    # MLP parameters: fc_1 + fc_2 + proj
    mlp_params = (width * intermediate_size) + (width * intermediate_size) + (intermediate_size * width)
    
    # RMSNorm parameters: 2 per block + 1 final norm
    norm_params_per_block = 2 * width
    
    # Total per block
    total_block_params = attn_params + mlp_params + norm_params_per_block
    
    # All layers (blocks)
    total_params = total_block_params * depth
    
    # Final LayerNorm and LM Head
    final_norm_params = width
    lm_head_params = width * vocab_size
    
    # Total model parameters
    return total_params + embedding_params + final_norm_params + lm_head_params


In [ ]:
min_width = 256
max_width = 8192
min_depth = 2
max_depth = 255

target_groups = [
    50e6,
    100e6,
    500e6,
    1e9,
    2e9,
    4e9,
    7e9,
]

tolerance = 0.05

# explicit shapes
vocab_size = 50304
head_size = 128

multiples_of_128 = list(range(min_width, max_width+1, 128))
depths = list(range(min_depth, max_depth+1))



all_configs = []
for target_param_count in target_groups:
    for n_embd in multiples_of_128:
        for n_layer in depths:

            try:
                assert n_embd%128 == 0, "width must be divisible by 128 for n_head and head_size"

                ## our choices

                n_head = n_embd / 128
                assert n_head % 1 == 0, "num heads must be an integer"
                n_head = int(n_head)
                assert n_head % 2 == 0, "num heads must be div by 2"
                assert n_head >= 2, "num heads must be at least 2"

                n_query_groups = n_head / 2
                assert n_query_groups % 1 == 0, "num query groups must be an integer"
                assert n_query_groups >= 1, "num query groups must be more than 0"

                func_param_count = param_counter(
                                width=n_embd, # swept
                                depth=n_layer, # swept
                                vocab_size=vocab_size, # fixed, pythia
                                n_head=n_head, # n_embd / head_size
                                head_size=128, # fixed, max size on ROCm
                                n_query_groups=int(n_head/2), # GQA at 2:1
                                intermediate_size=4*n_embd, # expansion factor of 4
                            )

                assert (func_param_count >= target_param_count * (1-tolerance)) and (func_param_count <= target_param_count * (1+tolerance))
                all_configs.append({'width': n_embd, 'depth': n_layer, 'param_count': func_param_count, 'target_param_count': target_param_count, 'width_depth_ratio': n_embd/n_layer})

            except Exception as e:
                pass # invalid combo


def filter_fn(all_configs, use_groups=True):

    all_configs.sort(key=lambda x: abs(x['param_count'] - x['target_param_count']))
    selected = []
    
    used_widths = set()
    used_depths = set()

    for group in target_groups:
        group_selected = []
        if use_groups:
            group_used_widths = set()
            group_used_depths = set()
        else:
            group_used_widths = used_widths
            group_used_depths = used_depths
        group_configs = [config for config in all_configs if config['target_param_count'] == group]
        group_configs.sort(key=lambda x: abs(x['param_count'] - x['target_param_count']))
        for config in group_configs:
            if config['width'] not in group_used_widths and config['depth'] not in group_used_depths:
                group_selected.append(config)
                group_used_widths.add(config['width'])
                group_used_depths.add(config['depth'])
        
        selected.extend(group_selected)

    return selected

# all_configs = filter_fn(all_configs, use_groups=True)
# all_configs = filter_fn(all_configs, use_groups=False)


df = pd.DataFrame(all_configs, columns=['width', 'depth', 'param_count', 'target_param_count', 'width_depth_ratio'])

# rename cols
df = df.rename(columns={
    'width': 'W',
    'depth': 'D',
    'width_depth_ratio': 'W/D',
    'param_count': 'N',
    'target_param_count': 'Tgt N',
})

# add points for special models

for model, params in special_models.items():
    print(model)
    print(params['Tgt N'])
    df = pd.concat([df, pd.DataFrame([{
        'W': params['W'],
        'D': params['D'],
        'W/D': params['W/D'],
        # 'N': params['N'],
        'Tgt N': 10e12,
        'label': params['Tgt N'],
    }])], ignore_index=True)

# sort the rows by the numerical value of the column 'Tgt N'
df = df.sort_values('Tgt N')
df["label"].unique()
df
# for any model that has a "label" reset Tgt N to the label
df.loc[df['label'].notnull(), 'Tgt N'] = df.loc[df['label'].notnull(), 'label']
# make a column, special that is a bool for whether the label is not null
df['Existing'] = df['label'].apply(lambda x: not pd.notnull(x))

df['N'] = df['N'].apply(lambda x: f"{x:,}")
# df['Tgt N'] = df['Tgt N'].apply(lambda x: f"{x:,}")
df['W/D'] = (df['W/D']).round()

df

In [ ]:
special_models.keys()

In [ ]:
# plot this in matplotlib

# shapes for specific groups
def get_marker_and_size_and_color(label):
    if label == 'Porian et al.' or label == "Chinchilla":
        return 'H', 60, 'purple'
    elif label in our_models.keys():
        return "*", 140 , "black" #'P'
    elif str(label).lower() in special_models.keys():
        return 's', 60, None
    else: # potential
        return 'o', 40, None


fig, ax = plt.subplots(figsize=(6, 6))

for label, df_group in list(df.groupby('Tgt N')):
    # for index, row in df_group.iterrows():
    marker, size, color = get_marker_and_size_and_color(label)
    if marker not in ['o', '*']: # possible and ours
        continue
    if label in industry_models:
        continue

    if type(label) == float:
        plus_minus_abs = label*tolerance 
        # pm_string = f"±{plus_minus_abs:.0e}".replace("+", "").replace("e0", "e")
        # label = f"{label:.0e}".replace("+", "").replace("e0", "e")
        # label = f"{label} {pm_string}"

        # represent as only Millions or Billions
        if label < 1e9:
            label = f"{label/1e6:.0f}M"
        else:
            label = f"{label/1e9:.0f}B"
        # add the ± in the same way
        if plus_minus_abs < 1e9:
            plus_minus_abs = f"{plus_minus_abs/1e6:.0f}M"
        else:
            plus_minus_abs = f"{plus_minus_abs/1e9:.0f}B"
        label = f"{label} ± {plus_minus_abs}"
        
    

        
    #     if "e6" in label:
    #         label = label.replace("e6", "M")
    #     elif "e9" in label:
    #         label = label.replace("e9", "B")
    # # print(label)
    

    if color is not None:
        ax.scatter(df_group['D'], df_group['W'], label=label, marker=marker, s=size, color=color)
    else:
        ax.scatter(df_group['D'], df_group['W'], label=label, marker=marker, s=size)

ax.grid(True, which="both", ls="--", alpha=0.5, zorder=0)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Depth')
ax.set_ylabel('Width')
ax.legend()

# remove our_models from legend
handles, labels = ax.get_legend_handles_labels()
# remove our_models from legend
handles = [h for h in handles]
labels = [l for l in labels]

single_version_of_our_label = None
for model in our_models.keys():
    if model in labels:
        idx = labels.index(model)
        handle = handles[idx]
        label = labels[idx]
        handles.pop(idx)
        labels.pop(idx)
        if single_version_of_our_label is None:
            single_version_of_our_label = f"Ours"

# add a single version of our models to the legend
if single_version_of_our_label is not None:
    handles.append(handle)
    labels.append(single_version_of_our_label)

ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncols=3, handlelength=0.5)

plt.show()

# save
fig.savefig(f'../figures/model_search_space.pdf', bbox_inches='tight')

In [ ]:
colors = {
    "Llama": "#1f77b4",      # Blue
    "Gemma": "#ff7f0e",      # Orange
    "MiniCPM": "#2ca02c",    # Green
    "Deepseek": "#d62728",   # Red
    "Phi": "#9467bd",        # Purple
    "Mistral": "#8c564b",    # Brown
    "TinyLlama": "#e377c2"   # Pink
}
label_to_category = {
    label: category
    for label, category in [
        ("Deepseek-67b", "Deepseek"),
        ("Deepseek-7b", "Deepseek"),
        ("Gemma-2-27b", "Gemma"),
        ("Gemma-2-2b", "Gemma"),
        ("Gemma-2-9b", "Gemma"),
        ("Gemma-2b", "Gemma"),
        ("Gemma-7b", "Gemma"),
        ("Llama-13b", "Llama"),
        ("Llama-2-13b", "Llama"),
        ("Llama-2-34b", "Llama"),
        ("Llama-2-70b", "Llama"),
        ("Llama-2-7b", "Llama"),
        ("Llama-3.1-405b", "Llama"),
        ("Llama-3.1-70b", "Llama"),
        ("Llama-3.1-8b", "Llama"),
        ("Llama-33b", "Llama"),
        ("Llama-65b", "Llama"),
        ("Llama-7b", "Llama"),
        ("MiniCPM-V-2-1.2b", "MiniCPM"),
        ("MiniCPM-V-2-2.4b", "MiniCPM"),
        ("Mistral-7b", "Mistral"),
        ("Phi-3.5-Mini", "Phi"),
        ("TinyLlama-1.1b", "TinyLlama")
    ]
}
label_to_color = {label: colors[category] for label, category in label_to_category.items()}

def clean_label(label):
    # Remove base category name using regex (e.g., "Deepseek-", "Llama-", etc.)
    return re.sub(r'^(Deepseek|Gemma|Llama|MiniCPM|Mistral|Phi|TinyLlama)-', '', label)


# shapes for specific groups
def get_marker_and_size_and_color(label):
    if label == "Chinchilla":
        return 'P', 90, 'purple'
    if label == 'Porian et al.':
        return '^', 90, 'forestgreen'
    elif label in our_models.keys():
        return "*", 140 , "black"
    elif str(label) in special_models.keys():
        return 's', 60, label_to_color[label]
    else: # potential
        return 'o', 40, None

fig, ax = plt.subplots(figsize=(6, 6))

# Store annotations for adjustment later
annotations = []
for label, df_group in list(df.groupby('Tgt N')):
    print(label)
    marker, size, color = get_marker_and_size_and_color(label)
    if marker == 'o': # ours and others
        continue

    if type(label) == float:
        plus_minus_abs = label * tolerance 
        if label < 1e9:
            label = f"{label / 1e6:.0f}M"
        else:
            label = f"{label / 1e9:.0f}B"
        if plus_minus_abs < 1e9:
            plus_minus_abs = f"{plus_minus_abs / 1e6:.0f}M"
        else:
            plus_minus_abs = f"{plus_minus_abs / 1e9:.0f}B"
        label = f"{label} ± {plus_minus_abs}"
    # print(label)
    zorder= 2
    if marker == '*':
        zorder= 3
    if label in ['Chinchilla', "Porian et al."]+list(our_models.keys()):
        ax.scatter(df_group['D'], df_group['W'], label=label, marker=marker, s=size, color=color, zorder=zorder)
    else:
        # If the marker is 's', annotate the points instead of adding to the legend
        if marker == 's':
            # for index, row in df_group.iterrows():
            #     # Annotate the points
            #     annotation = ax.annotate(
            #         clean_label(label), 
            #         (row['D'], row['W']),  # Annotate exactly at the data point
            #         textcoords="offset points",
            #         xytext=(5, 5),  # Slight offset for better visibility
            #         ha='center',
            #         arrowprops=dict(arrowstyle="-", color='gray', lw=0.5),  # Use arrowprops to draw an arrow
            #         zorder=1
            #     )
            #     annotations.append(annotation)  # Store annotations for adjustment
            ax.scatter(df_group['D'], df_group['W'], marker=marker, s=size, color=color, zorder=zorder, label=label_to_category[label])
        else:
            ax.scatter(df_group['D'], df_group['W'], label=label, marker=marker, s=size, zorder=zorder)
# Adjust the text and arrow positions to avoid overlap and correct misplacement
adjust_text(
    annotations, 
    ax=ax, 
    expand_text=(1.2, 1.4), 
    arrowprops=dict(arrowstyle="-", color='gray', lw=0.0),  # Redraw arrowprops to adjust after text moves
    zorder=1
)

# grid
ax.grid(True, which="major", ls="--", alpha=0.5, zorder=0)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel('Depth')
ax.set_ylabel('Width')

ax.legend()

# remove our_models from legend
handles, labels = ax.get_legend_handles_labels()
handles = [h for h in handles]
labels = [l for l in labels]

single_version_of_our_label = None
for model in our_models.keys():
    if model in labels:
        idx = labels.index(model)
        handle = handles[idx]
        label = labels[idx]
        handles.pop(idx)
        labels.pop(idx)
        if single_version_of_our_label is None:
            single_version_of_our_label = "Ours"

# add a single version of our models to the legend
if single_version_of_our_label is not None:
    handles.append(handle)
    labels.append(single_version_of_our_label)


by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc='upper center', bbox_to_anchor=(0.5, -0.15), ncols=3, handlelength=0.5)

plt.show()
fig.savefig(f'../figures/model_search_space_with_commerical_models.pdf', bbox_inches='tight')
